In [ ]:
import sys, os
os.environ['OMP_NUM_THREADS']='1'
sys.path.insert(0, f'{os.environ["HOME"]}/git/ImageD11')

In [ ]:
import ImageD11.sinograms.point_by_point
import ImageD11.sinograms.dataset
import ImageD11.sinograms.properties
from ImageD11.blobcorrector import eiger_spatial
import ImageD11.columnfile
import pylab as pl, numpy as np

In [ ]:
!pwd

In [ ]:
parname = '/data/visitor/ihma423/id11/20231205/PROCESSED_DATA/S7_deformed/Al_fcc_2.par'
SparsePixelFolder = '/data/visitor/ihma423/id11/20231205/PROCESSED_DATA/SparsePixels_NewMask/'
sample = 'S9_355C_10min_3rd'
dset_prefix = 'sliceZ_-360um'
cf_postfix = '4rha'

z = 5
if 1:
    
    dsname = os.path.join(SparsePixelFolder, 'ds_' + sample + '_' + dset_prefix + '_' + str(z) + '.h5')
    pksname = os.path.join(SparsePixelFolder, 'pks_' + sample + '_' + dset_prefix + '_' + str(z) + '.h5')
    outname = os.path.join(SparsePixelFolder, sample + '_' + dset_prefix + '_' + str(z) + '_sparse.h5')                  
    cf_h5 = os.path.join(os.getcwd(), sample + '_' + dset_prefix + '_' + str(z) + '_' + cf_postfix + '.h5')               
    pbp_out = os.path.join(os.getcwd(), sample + '_' + dset_prefix + '_' + str(z) + '_' + cf_postfix + '.pbp')

In [ ]:
pk = ImageD11.sinograms.properties.pks_table.load(pksname)
dset = ImageD11.sinograms.dataset.load(dsname)
es = eiger_spatial(dxfile='/data/id11/nanoscope/Eiger/e2dx_E-08-0173_20231127.edf',
                   dyfile='/data/id11/nanoscope/Eiger/e2dy_E-08-0173_20231127.edf' )
cf = ImageD11.columnfile.colfile_from_dict( es(pk.pk2d(dset.omega, dset.dty)) )
cf.filter(cf.Number_of_pixels > 5)

In [ ]:
p= ImageD11.sinograms.point_by_point.PBP(
    parname,
    dset,
    hkl_tol=0.01,
        fpks=0.7,
        ds_tol=0.01,
        etacut=0.1,
        ifrac=1./300,
        cosine_tol=np.cos(np.radians(90 - 0.1)),
        y0=0,
        symmetry="cubic",
        foridx=[0, 1, 2, 4, 5],
        forgen=[5, 1],
        uniqcut=0.75)

In [ ]:
p.setpeaks(cf)

In [ ]:
f, ax = p.iplot()

In [ ]:
p.point_by_point('grainsout2.pbp', nprocs=1, debugpoints=[(10,10),], loglevel=0)
!tail grainsout2.pbp

In [ ]:
p.point_by_point('grainsout_all.pbp', loglevel=3)